In [1]:
# Solo para validar que relaciones declaradas en archivo de ppto, se están cumpliendo

In [2]:
import sys
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
mode = 'PC'
paises = ['MX'] #['AR', 'MX', 'BR', 'PE', 'CO', 'CL', 'UY']

#nombre_sheets_ppto = 'Copy of Presupuesto Retail Digital 2024'
nombre_sheets_ppto = 'Presupuesto Retail Digital 2025'

In [4]:
carpeta_modulos = '../../Modulos/'  #/home/data/python/performance_automation/Modulos
carpeta_modulos_propios = '../Modulos/' 
carpeta_resultados = '../Resultados/'
carpeta_input = '../Inputs/'

In [5]:
if mode == 'VM': # Esto tiene que ir en funcion generar dag
    carpeta_modulos = '/home/data/python/performance_automation/Modulos/'

sys.path.insert(0, carpeta_modulos)
import funciones_transversales as tf

### Metas resumen

In [6]:
metricas = {'VENTA_NETA': [4, 6], 'ORDENES_NETAS': [11, 13], 'TP_NETO': [18, 20], 'TFYC': [38, 40], 'IMPUESTO': [42, 42], 'VENTA_COLOCADA': [45, 47], 'ORDENES_COLOCADAS': [50, 52], 'TP_COLOCADO': [55, 57], 'VISITAS': [60, 62], 'TC': [70, 72]}
            
            
            #'VENTA_COLOCADA': [41, 43], 'ORDENES': [46, 48], , 'INVERSION': [62, 62], 'TFYC': [38, 38], 'IMPUESTO': [39, 39], 'PORCENTAJE_TRAFICO_ORGANICO': [60, 60]}


# declaracion explícita de las filas (se ajustan solas) [incluidos desde y hasta]
#  Ordenes son colocadas

In [7]:
df_resumen_metas = pd.DataFrame()
for pais in paises:
    print(pais)
    df = tf.lectura_segura_Gsheets(nombre_sheets_ppto, pais)
    
    df_pais = pd.DataFrame()
    for metrica in metricas:
        print('     ', metrica)
        [i_0, i_f] = metricas[metrica]
        i_0, i_f = i_0 - 2, i_f - 1
        
        #print(metrica)
        
        #print('DF0')
        #display(df.iloc[i_0:i_f])
        df_metrica = df.iloc[i_0:i_f, 1:15].reset_index(drop = True)
        
        
        #display(df_metrica)
        
        df_metrica.columns = ['CANAL'] + [m for m in range(1, 13)] + ['TOTAL']
        df_metrica['CANAL'] = np.where(df_metrica['CANAL'] == 'APP SODIMAC', 'APP', np.where(df_metrica['CANAL'] == 'WEB SODIMAC', 'WEB', np.where(df_metrica['CANAL'].isin(['FCOM / LINIO', 'FCOM']), 'FCOM', '')))
        
        #display(df_metrica)

        for m in list(range(1, 13)) + ['TOTAL']:
            
            #print(m)
            #display(df_metrica[m])
            df_metrica[m] = df_metrica[m].str.replace(',', '')
            df_metrica[m] = df_metrica[m].str.replace('%', '')
            df_metrica[m] = df_metrica[m].str.replace('-', '')
            df_metrica[m] = df_metrica[m].replace('', 0).astype(float)
            df_metrica[m] = df_metrica[m].fillna(0)
            
            if metrica in ['TFYC', 'IMPUESTO', 'TC']:
                df_metrica[m] = df_metrica[m] / 100
        
        #display(df_metrica)
        
        df_metrica.insert(0, 'METRICA', metrica)
        df_metrica.insert(0, 'PAIS', pais)
        
        df_pais = pd.concat([df_pais, df_metrica], axis = 0, ignore_index = True)
    
    df_resumen_metas = pd.concat([df_resumen_metas, df_pais], axis = 0, ignore_index = True)

print('----------------------------------------------') 
display(df_resumen_metas)
#sys.exit()
 
df_resumen_metas_general = df_resumen_metas[~df_resumen_metas['METRICA'].isin(['IMPUESTO'])]
df_resumen_metas_otros = df_resumen_metas[df_resumen_metas['METRICA'].isin(['IMPUESTO'])]

# Pivotear y meltear + eliminar ceros
# Las columnas 1..12 son los meses.  Crear el campo mes (meltear)
df_resumen_metas_general = pd.melt(df_resumen_metas_general, id_vars = ['PAIS', 'METRICA', 'CANAL'], var_name = 'MES', value_name = 'VALOR')

display(df_resumen_metas_general)
# pivotear el campo METRICA
df_resumen_metas_general = df_resumen_metas_general.pivot_table(index = ['PAIS', 'CANAL', 'MES'], columns = 'METRICA', values = 'VALOR').reset_index()
#df_resumen_metas_general = df_resumen_metas_general[df_resumen_metas_general['VENTA_COLOCADA'] > 0].reset_index(drop = True) # bloqueadio el 240213
#df_resumen_metas_general.to_csv(f'{carpeta_input}/data_presupuesto/0_df_resumen_metas_general.csv', index = False, sep = ';', decimal = ',')
display(df_resumen_metas_general)

df_resumen_metas_otros = df_resumen_metas_otros.drop(columns = 'CANAL')
df_resumen_metas_otros = pd.melt(df_resumen_metas_otros, id_vars = ['PAIS', 'METRICA'], var_name = 'MES', value_name = 'VALOR')
df_resumen_metas_otros = df_resumen_metas_otros.pivot_table(index = ['PAIS', 'MES'], columns = 'METRICA', values = 'VALOR').reset_index()
#df_resumen_metas_otros.to_csv(f'{carpeta_input}/data_presupuesto/1_df_resumen_metas_otros.csv', index = False, sep = ';', decimal = ',')
display(df_resumen_metas_otros)
df_resumen_metas_general = df_resumen_metas_general.merge(df_resumen_metas_otros, how = 'left', on = ['PAIS', 'MES'])
df_resumen_metas_general

      VENTA_NETA
      ORDENES_NETAS
      TP_NETO
      TFYC
      IMPUESTO
      VENTA_COLOCADA
      ORDENES_COLOCADAS
      TP_COLOCADO
      VISITAS
      TC
----------------------------------------------


,PAIS,METRICA,CANAL,1,2,3,4,5,6,7,8,9,10,11,12,TOTAL
0,MX,VENTA_NETA,APP,4.571249e+06,4.008125e+06,5.531854e+06,3.643750e+06,9.937500e+06,5.101250e+06,4.505000e+06,5.300000e+06,4.438750e+06,3.776250e+06,1.060000e+07,4.836250e+06,6.624998e+07
1,MX,VENTA_NETA,WEB,1.078815e+07,9.459175e+06,1.305518e+07,8.599250e+06,2.345250e+07,1.203895e+07,1.063180e+07,1.250800e+07,1.047545e+07,8.911950e+06,2.501608e+07,1.141355e+07,1.563500e+08
2,MX,VENTA_NETA,FCOM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,MX,ORDENES_NETAS,APP,1.394000e+03,1.196000e+03,1.627000e+03,1.056000e+03,2.966000e+03,1.523000e+03,1.287000e+03,1.514000e+03,1.287000e+03,1.144000e+03,2.944000e+03,1.422000e+03,1.936200e+04
4,MX,ORDENES_NETAS,WEB,3.480000e+03,3.003000e+03,4.145000e+03,2.687000e+03,7.445000e+03,3.884000e+03,3.222000e+03,3.790000e+03,3.223000e+03,2.875000e+03,7.358000e+03,3.512000e+03,4.862300e+04
5,MX,ORDENES_NETAS,FCOM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,MX,TP_NETO,APP,3.280000e+03,3.350000e+03,3.400000e+03,3.450000e+03,3.350000e+03,3.350000e+03,3.500000e+03,3.500000e+03,3.450000e+03,3.300000e+03,3.600000e+03,3.400000e+03,3.422000e+03
7,MX,TP_NETO,WEB,3.100000e+03,3.150000e+03,3.150000e+03,3.200000e+03,3.150000e+03,3.100000e+03,3.300000e+03,3.300000e+03,3.250000e+03,3.100000e+03,3.400000e+03,3.250000e+03,3.216000e+03
8,MX,TP_NETO,FCOM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,MX,TFYC,APP,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01,2.400000e-01


,PAIS,METRICA,CANAL,MES,VALOR
0,MX,VENTA_NETA,APP,1,4.571249e+06
1,MX,VENTA_NETA,WEB,1,1.078815e+07
2,MX,VENTA_NETA,FCOM,1,0.000000e+00
3,MX,ORDENES_NETAS,APP,1,1.394000e+03
4,MX,ORDENES_NETAS,WEB,1,3.480000e+03
...,...,...,...,...,...
346,MX,VISITAS,WEB,TOTAL,1.689911e+07
347,MX,VISITAS,FCOM,TOTAL,0.000000e+00
348,MX,TC,APP,TOTAL,9.400000e-03
349,MX,TC,WEB,TOTAL,3.400000e-03


METRICA,PAIS,CANAL,MES,ORDENES_COLOCADAS,ORDENES_NETAS,TC,TFYC,TP_COLOCADO,TP_NETO,VENTA_COLOCADA,VENTA_NETA,VISITAS
0,MX,APP,1,1626.0,1394.0,0.0122,0.24,4291.0,3280.0,6977170.0,4571249.0,133189.0
1,MX,APP,2,1395.0,1196.0,0.0118,0.24,4385.0,3350.0,6117664.0,4008125.0,118387.0
2,MX,APP,3,1900.0,1627.0,0.0109,0.24,4444.0,3400.0,8443356.0,5531854.0,174482.0
3,MX,APP,4,1244.0,1056.0,0.0094,0.24,4471.0,3450.0,5561513.0,3643750.0,132685.0
4,MX,APP,5,3450.0,2966.0,0.0106,0.24,4396.0,3350.0,15167763.0,9937500.0,326106.0
5,MX,APP,6,1792.0,1523.0,0.0093,0.24,4345.0,3350.0,7786118.0,5101250.0,193039.0
6,MX,APP,7,1493.0,1287.0,0.0077,0.24,4606.0,3500.0,6876053.0,4505000.0,192839.0
7,MX,APP,8,1754.0,1514.0,0.0074,0.24,4612.0,3500.0,8089474.0,5300000.0,237968.0
8,MX,APP,9,1505.0,1287.0,0.0089,0.24,4502.0,3450.0,6774934.0,4438750.0,169539.0
9,MX,APP,10,1343.0,1144.0,0.0071,0.24,4292.0,3300.0,5763750.0,3776250.0,189660.0


METRICA,PAIS,MES,IMPUESTO
0,MX,1,0.16
1,MX,2,0.16
2,MX,3,0.16
3,MX,4,0.16
4,MX,5,0.16
5,MX,6,0.16
6,MX,7,0.16
7,MX,8,0.16
8,MX,9,0.16
9,MX,10,0.16


METRICA,PAIS,CANAL,MES,ORDENES_COLOCADAS,ORDENES_NETAS,TC,TFYC,TP_COLOCADO,TP_NETO,VENTA_COLOCADA,VENTA_NETA,VISITAS,IMPUESTO
0,MX,APP,1,1626.0,1394.0,0.0122,0.24,4291.0,3280.0,6977170.0,4571249.0,133189.0,0.16
1,MX,APP,2,1395.0,1196.0,0.0118,0.24,4385.0,3350.0,6117664.0,4008125.0,118387.0,0.16
2,MX,APP,3,1900.0,1627.0,0.0109,0.24,4444.0,3400.0,8443356.0,5531854.0,174482.0,0.16
3,MX,APP,4,1244.0,1056.0,0.0094,0.24,4471.0,3450.0,5561513.0,3643750.0,132685.0,0.16
4,MX,APP,5,3450.0,2966.0,0.0106,0.24,4396.0,3350.0,15167763.0,9937500.0,326106.0,0.16
5,MX,APP,6,1792.0,1523.0,0.0093,0.24,4345.0,3350.0,7786118.0,5101250.0,193039.0,0.16
6,MX,APP,7,1493.0,1287.0,0.0077,0.24,4606.0,3500.0,6876053.0,4505000.0,192839.0,0.16
7,MX,APP,8,1754.0,1514.0,0.0074,0.24,4612.0,3500.0,8089474.0,5300000.0,237968.0,0.16
8,MX,APP,9,1505.0,1287.0,0.0089,0.24,4502.0,3450.0,6774934.0,4438750.0,169539.0,0.16
9,MX,APP,10,1343.0,1144.0,0.0071,0.24,4292.0,3300.0,5763750.0,3776250.0,189660.0,0.16


## Relación 1: Venta neta y colocada

In [8]:
df_resumen_metas_general['VENTA_COLOCADA_2'] = df_resumen_metas_general['VENTA_NETA'] * (1 + df_resumen_metas_general['IMPUESTO']) / (1 - df_resumen_metas_general['TFYC'])
df_resumen_metas_general['DELTA_VN_VC'] = np.where(df_resumen_metas_general['VENTA_COLOCADA_2'] + df_resumen_metas_general['VENTA_COLOCADA'] == 0, 0, abs(df_resumen_metas_general['VENTA_COLOCADA_2'] - df_resumen_metas_general['VENTA_COLOCADA']) / df_resumen_metas_general['VENTA_COLOCADA'])
df_resumen_metas_general['RELACION DIRECTA VN VC'] = df_resumen_metas_general['DELTA_VN_VC'] <= 10 ** -3
#df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['VENTA_COLOCADA_2', 'DELTA_VN_VC'])
df_resumen_metas_general.head()

METRICA,PAIS,CANAL,MES,ORDENES_COLOCADAS,ORDENES_NETAS,TC,TFYC,TP_COLOCADO,TP_NETO,VENTA_COLOCADA,VENTA_NETA,VISITAS,IMPUESTO,VENTA_COLOCADA_2,DELTA_VN_VC,RELACION DIRECTA VN VC
0,MX,APP,1,1626.0,1394.0,0.0122,0.24,4291.0,3280.0,6977170.0,4571249.0,133189.0,0.16,6.977170e+06,6.789059e-08,True
1,MX,APP,2,1395.0,1196.0,0.0118,0.24,4385.0,3350.0,6117664.0,4008125.0,118387.0,0.16,6.117664e+06,7.742894e-08,True
2,MX,APP,3,1900.0,1627.0,0.0109,0.24,4444.0,3400.0,8443356.0,5531854.0,174482.0,0.16,8.443356e+06,1.246698e-08,True
3,MX,APP,4,1244.0,1056.0,0.0094,0.24,4471.0,3450.0,5561513.0,3643750.0,132685.0,0.16,5.561513e+06,2.839061e-08,True
4,MX,APP,5,3450.0,2966.0,0.0106,0.24,4396.0,3350.0,15167763.0,9937500.0,326106.0,0.16,1.516776e+07,1.040989e-08,True


## Relación 2: TP, Venta y ordenes

In [9]:
# Colocado

df_resumen_metas_general['VENTA_COLOCADA_2'] = df_resumen_metas_general['TP_COLOCADO'] * df_resumen_metas_general['ORDENES_COLOCADAS']
df_resumen_metas_general['DELTAS_VC'] = np.where((df_resumen_metas_general['VENTA_COLOCADA'] == 0) & (df_resumen_metas_general['VENTA_COLOCADA_2'] == 0), 0, abs(df_resumen_metas_general['VENTA_COLOCADA_2'] - df_resumen_metas_general['VENTA_COLOCADA']) / df_resumen_metas_general['VENTA_COLOCADA'])
df_resumen_metas_general['RELACION TPC OC y VC'] = df_resumen_metas_general['DELTAS_VC'] <= 2 * (10 ** -2)
df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['VENTA_COLOCADA_2', 'DELTAS_VC'])

# Neto

df_resumen_metas_general['VENTA_NETA_2'] = df_resumen_metas_general['TP_NETO'] * df_resumen_metas_general['ORDENES_NETAS']
df_resumen_metas_general['DELTAS_VN'] = np.where((df_resumen_metas_general['VENTA_NETA'] == 0) & (df_resumen_metas_general['VENTA_NETA_2'] == 0), 0, abs(df_resumen_metas_general['VENTA_NETA_2'] - df_resumen_metas_general['VENTA_NETA']) / df_resumen_metas_general['VENTA_NETA'])
#display(df_resumen_metas_general)
df_resumen_metas_general['RELACION TPN ON y VN'] = df_resumen_metas_general['DELTAS_VN'] <= 2 * (10 ** -2)
df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['VENTA_NETA_2', 'DELTAS_VN'])
df_resumen_metas_general
#df_resumen_metas_general

METRICA,PAIS,CANAL,MES,ORDENES_COLOCADAS,ORDENES_NETAS,TC,TFYC,TP_COLOCADO,TP_NETO,VENTA_COLOCADA,VENTA_NETA,VISITAS,IMPUESTO,DELTA_VN_VC,RELACION DIRECTA VN VC,RELACION TPC OC y VC,RELACION TPN ON y VN
0,MX,APP,1,1626.0,1394.0,0.0122,0.24,4291.0,3280.0,6977170.0,4571249.0,133189.0,0.16,6.789059e-08,True,True,True
1,MX,APP,2,1395.0,1196.0,0.0118,0.24,4385.0,3350.0,6117664.0,4008125.0,118387.0,0.16,7.742894e-08,True,True,True
2,MX,APP,3,1900.0,1627.0,0.0109,0.24,4444.0,3400.0,8443356.0,5531854.0,174482.0,0.16,1.246698e-08,True,True,True
3,MX,APP,4,1244.0,1056.0,0.0094,0.24,4471.0,3450.0,5561513.0,3643750.0,132685.0,0.16,2.839061e-08,True,True,True
4,MX,APP,5,3450.0,2966.0,0.0106,0.24,4396.0,3350.0,15167763.0,9937500.0,326106.0,0.16,1.040989e-08,True,True,True
5,MX,APP,6,1792.0,1523.0,0.0093,0.24,4345.0,3350.0,7786118.0,5101250.0,193039.0,0.16,5.407735e-08,True,True,True
6,MX,APP,7,1493.0,1287.0,0.0077,0.24,4606.0,3500.0,6876053.0,4505000.0,192839.0,0.16,5.358031e-08,True,True,True
7,MX,APP,8,1754.0,1514.0,0.0074,0.24,4612.0,3500.0,8089474.0,5300000.0,237968.0,0.16,3.903708e-08,True,True,True
8,MX,APP,9,1505.0,1287.0,0.0089,0.24,4502.0,3450.0,6774934.0,4438750.0,169539.0,0.16,3.107430e-08,True,True,True
9,MX,APP,10,1343.0,1144.0,0.0071,0.24,4292.0,3300.0,5763750.0,3776250.0,189660.0,0.16,0.000000e+00,True,True,True


### Relación TPN y TPC

In [10]:
#df_resumen_metas_general['TP_COLOCADO_2'] = df_resumen_metas_general['TP_NETO'] * (1 + df_resumen_metas_general['IMPUESTO'])
#df_resumen_metas_general['DELTAS_TPC'] = abs(df_resumen_metas_general['TP_COLOCADO'] - df_resumen_metas_general['TP_COLOCADO_2']) / df_resumen_metas_general['TP_COLOCADO']
#df_resumen_metas_general['RELACION TPC TPN'] = df_resumen_metas_general['DELTAS_TPC'] <= 3 * (10 ** -2) # 3%
#df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['TP_COLOCADO_2', 'DELTAS_TPC'])


### Relación Ordenes N y Ordenes C

In [11]:
#df_resumen_metas_general['ORDENES_COLOCADAS_2'] = df_resumen_metas_general['ORDENES_NETAS'] / (1 - df_resumen_metas_general['TFYC'])
#df_resumen_metas_general['DELTAS_ORDENES_COLOCADAS'] = abs(df_resumen_metas_general['ORDENES_COLOCADAS_2'] - df_resumen_metas_general['ORDENES_COLOCADAS']) / df_resumen_metas_general['ORDENES_COLOCADAS']
#df_resumen_metas_general['RELACION_ORDENES_COLOCADAS'] = df_resumen_metas_general['DELTAS_ORDENES_COLOCADAS'] <= 3 * (10 ** -2) # 3%
#df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['DELTAS_ORDENES_COLOCADAS', 'ORDENES_COLOCADAS_2'])
#df_resumen_metas_general

### Relación visitas, TC y ordenes colocadas

In [12]:
df_resumen_metas_general['VISITAS_2'] = np.where(df_resumen_metas_general['TC'] == 0, 0, df_resumen_metas_general['ORDENES_COLOCADAS'] / df_resumen_metas_general['TC'])
df_resumen_metas_general['DELTAS_VISITAS'] = np.where((df_resumen_metas_general['VISITAS'] == 0) & (df_resumen_metas_general['VISITAS_2'] == 0), 0, abs(df_resumen_metas_general['VISITAS_2'] - df_resumen_metas_general['VISITAS']) / df_resumen_metas_general['VISITAS'])
df_resumen_metas_general['RELACION_VISITAS'] = df_resumen_metas_general['DELTAS_VISITAS'] <= 2 * (10 ** -2) # 2%
#display(df_resumen_metas_general)
df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['DELTAS_VISITAS', 'VISITAS_2'])
#df_resumen_metas_general = df_resumen_metas_general.drop(columns = ['TP_COLOCADO', 'TP_NETO', 'VENTA_COLOCADA', 'VENTA_NETA'])
#df_resumen_metas_general

### Primera revisión: Consistencia numérica

In [13]:
diccionario_metricas = {'RELACION DIRECTA VN VC': ['VENTA_NETA', 'TFYC', 'IMPUESTO', 'VENTA_COLOCADA'],
 'RELACION TPC OC y VC': ['TP_COLOCADO', 'ORDENES_COLOCADAS', 'VENTA_COLOCADA'],
 #'RELACION TPC TPN': ['TP_COLOCADO', 'TP_NETO', 'IMPUESTO'],
 'RELACION TPN ON y VN': ['TP_NETO', 'ORDENES_NETAS', 'VENTA_NETA'],
 #'RELACION_ORDENES_COLOCADAS': ['ORDENES_NETAS', 'ORDENES_COLOCADAS', 'TFYC'],
 'RELACION_VISITAS': ['ORDENES_COLOCADAS', 'TC', 'VISITAS']}

In [14]:
dic_canales = {'AR': 'FCOM', 'BR': 'FCOM', 'CL': '', 'PE': '', 'CO': 'FCOM', 'UY': 'FCOM', 'MX': 'FCOM'}

In [15]:
status1 = True
for relacion in diccionario_metricas:
    
    
    df_errores = df_resumen_metas_general[~df_resumen_metas_general[relacion]].reset_index(drop = True)
    
    df_errores = df_errores[df_errores['CANAL'] != dic_canales[pais]]
    if len(df_errores) > 0:
        
        status1 = False
        print(relacion)
        
        if relacion == 'RELACION DIRECTA VN VC':
            print ('NO CONSIDERAR SI TF&C ESTÁ DECLARADA POR CANAL (ej, en BR, CL)')
        display(df_errores[['PAIS', 'CANAL', 'MES'] + diccionario_metricas[relacion]])
        
if status1:
    print('No hay errores en las relaciones')

No hay errores en las relaciones


### Totales (Revisión 2)

In [16]:
status2 = True
for metrica in ['ORDENES_COLOCADAS', 'ORDENES_NETAS', 'VENTA_COLOCADA', 'VENTA_NETA', 'VISITAS']:
    df_metrica = df_resumen_metas_general[['PAIS', 'CANAL', 'MES', metrica]]
    # MES como coumna
    df_metrica = df_metrica.pivot_table(index = ['PAIS', 'CANAL'], columns = 'MES', values = metrica).reset_index()
    df_metrica['TOTAL2'] = df_metrica[list(range(1, 13))].sum(axis = 1)
    df_metrica['DELTAS'] = np.where((df_metrica['TOTAL'] == 0) & (df_metrica['TOTAL2'] == 0), 0, abs(df_metrica['TOTAL2'] - df_metrica['TOTAL']) / df_metrica['TOTAL'])
    ##display(df_metrica)
    df_metrica['RELACION_TOTAL'] = df_metrica['DELTAS'] <= 10 ** -3
    df_metrica = df_metrica.drop(columns = ['DELTAS', 'TOTAL2'])
    
    df_errores = df_metrica[~df_metrica['RELACION_TOTAL']].reset_index(drop = True)
    if len(df_errores) > 0:
        status2 = False
        print(metrica)
        display(df_errores)
    #sys.exit()

if status2:
    print('No hay errores en las relaciones')

No hay errores en las relaciones


In [17]:
print('Proceso Completo')

Proceso Completo
